добавим импорты

In [ ]:
!pip install fastapi uvicorn[standard] nest-asyncio pyngrok scikit-learn joblib
!pip install python-multipart

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 1.6 MB/s eta 0:00:00


In [ ]:
import nest_asyncio
from pyngrok import ngrok
from pyngrok.conf import PyngrokConfig
import joblib
import numpy as np
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI, HTTPException, UploadFile, File
from pydantic import BaseModel
from typing import List

In [ ]:
# 1. Настройка ngrok с вашим токеном
ngrok.set_auth_token("2wSJ5VaAqlgK9jjKEjqXJIzJjq0_iyrjVymxXRqBZi4Q7pjy")

# 2. Инициализация FastAPI
app = FastAPI()

# Разрешить CORS (для тестов)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# 3. Загрузка модели (пример)
# Загрузите файл модели через левую панель Colab
model = joblib.load("gb_model.joblib")
model

FileNotFoundError: [Errno 2] No such file or directory: 'gb_model.joblib'

In [ ]:
# 4. API Endpoint
@app.get("/")
async def root():
    return {
        "message": "Server is running",
        "endpoints": {
            "/predict": "POST with JSON {'features': [...]}",
            "/predict_from_file": "POST with text file containing 764 comma-separated values"
        }
    }

@app.post("/predict_from_file")
async def predict_from_file(file: UploadFile = File(...)):
    try:
        print(f"Debug: Received file - filename: {file.filename}, content-type: {file.content_type}")

        content = await file.read()
        features_str = content.decode("utf-8").strip()

        print(f"Debug: Received file content (first 100 chars): {features_str[:100]}")
        print(f"Debug: File content length: {len(features_str)}")

        features = []
        for x in features_str.split(","):
            try:
                val = x.strip()
                if not val:  # Skip empty values
                    continue
                features.append(float(val))
            except ValueError:
                raise HTTPException(400, f"Invalid value in file: '{x}'")

        print(f"Debug: Parsed {len(features)} features")

        if len(features) != 764:
            raise HTTPException(400,
                f"Expected 764 features, got {len(features)}. First 5: {features[:5]}")

        input_data = np.array(features).reshape(1, -1)
        proba = model.predict_proba(input_data)[0, 1]

        return {
            "status": "success",
            "prediction": float(proba),
            "features_processed": len(features)
        }

    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(500, f"Internal server error: {str(e)}")

In [ ]:
class FeaturesRequest(BaseModel):
    features: List[float]

@app.post("/predict")
async def predict(request: FeaturesRequest):
    try:
        print(f"Received features (first 5): {request.features[:5]}")
        print(f"Total features received: {len(request.features)}")

        if len(request.features) != 764:
            error_msg = f"Expected 764 features, got {len(request.features)}"
            print(error_msg)
            raise HTTPException(status_code=400, detail=error_msg)

        input_data = np.array(request.features, dtype=np.float32).reshape(1, -1)
        print(f"Input array shape: {input_data.shape}")

        proba = model.predict_proba(input_data)[0, 1]  # Убедитесь, что model существует
        print(f"Prediction probability: {proba}")

        return {
            "status": "success",
            "prediction": float(proba),
            "features_received": len(request.features)
        }

    except Exception as e:
        print(f"Error: {str(e)}")
        raise HTTPException(500, detail=str(e))

In [ ]:
# 5. Запуск сервера
if __name__ == "__main__":
    nest_asyncio.apply()

    # Открываем туннель
    public_url = ngrok.connect(8000).public_url
    print("\n🔥 Сервер доступен по URL:", public_url)
    print("📚 Документация API:", f"{public_url}/docs\n")

    # Запускаем FastAPI
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


🔥 Сервер доступен по URL: https://ea9e-35-194-70-112.ngrok-free.app
📚 Документация API: https://ea9e-35-194-70-112.ngrok-free.app/docs



INFO:     Started server process [1201]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Received features (first 5): [-1.364664365406165, 1.0513983216939051, -1.2612337651151286, -1.2181390072093163, 0.023387432380440762]
Total features received: 765
Expected 764 features, got 765
Error: 400: Expected 764 features, got 765
INFO:     5.189.111.215:0 - "POST /predict HTTP/1.1" 500 Internal Server Error
Received features (first 5): [-1.364664365406165, 1.0513983216939051, -1.2612337651151286, -1.2181390072093163, 0.023387432380440762]
Total features received: 764
Input array shape: (1, 764)
Prediction probability: 5.1594549843350496e-08
INFO:     5.189.111.215:0 - "POST /predict HTTP/1.1" 200 OK


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


Received features (first 5): [-1.364664365406165, 1.0513983216939051, -1.2612337651151286, -1.2181390072093163, 0.023387432380440762]
Total features received: 764
Input array shape: (1, 764)
Prediction probability: 5.1594549843350496e-08
INFO:     5.189.111.215:0 - "POST /predict HTTP/1.1" 200 OK


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


Received features (first 5): [-0.8607495285461352, -0.7126339504146454, -0.31005299087674104, -0.6241783457236083, -0.31616839726294566]
Total features received: 764
Input array shape: (1, 764)
Prediction probability: 0.9999999268084336
INFO:     5.189.111.215:0 - "POST /predict HTTP/1.1" 200 OK


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


Received features (first 5): [-0.8607495285461352, -0.7126339504146454, -0.31005299087674104, -0.6241783457236083, -0.31616839726294566]
Total features received: 764
Input array shape: (1, 764)
Prediction probability: 0.9999999268084336
INFO:     5.189.111.215:0 - "POST /predict HTTP/1.1" 200 OK


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1201]


In [ ]:
# server.py
from fastapi import FastAPI, HTTPException, UploadFile, File
from fastapi.middleware.cors import CORSMiddleware
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
from typing import List
import io
from PIL import Image, ImageFilter
import nest_asyncio
from pyngrok import ngrok
import joblib
import numpy as np

# 1. Настройка ngrok с вашим токеном
ngrok.set_auth_token("2wSJ5VaAqlgK9jjKEjqXJIzJjq0_iyrjVymxXRqBZi4Q7pjy")

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Пример загрузки модели для числовых предсказаний (если нужно)
# model = joblib.load('model.joblib')

@app.post("/predict_from_file")
async def predict_from_file(file: UploadFile = File(...)):
    try:
        # 1) Читаем загруженный файл как изображение
        contents = await file.read()
        img = Image.open(io.BytesIO(contents)).convert("RGB")

        # 2) Обработка через ИИ (здесь пример — размытие и конвертация в оттенки серого)
        #    Замените на ваш ML/Deep Learning pipeline
        processed = img.filter(ImageFilter.GaussianBlur(radius=2))
        processed = processed.convert("L").convert("RGB")

        # 3) Сохраняем результат в байтовый буфер
        buf = io.BytesIO()
        processed.save(buf, format="JPEG")
        buf.seek(0)

        # 4) Отправляем обратно клиенту
        return StreamingResponse(buf, media_type="image/jpeg")

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    nest_asyncio.apply()
    public_url = ngrok.connect(8000).public_url
    print("Server running at", public_url)
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [161]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Server running at https://8b31-34-53-54-83.ngrok-free.app
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0:49:acff:2f01:0 - "POST /predict_from_file HTTP/1.1" 200 OK
INFO:     2a00:1fa2:4226:cb86:0: